# Gevent模块

Gevent 是一个第三方库,可以轻松通过gevent实现并发同步或异步编程,在gevent中用到的主要模式是Greenlet, 它是以C扩展模块形式接入Python的轻量级协程。 

Greenlet全部运行在主程序操作系统进程的内部,但它们被协作式地调度。

In [4]:
'''
#用法
创建一个协程对象g1,spawn括号内第一个参数是函数名,如eat,后面可以有多个参数,可以是位置实参或关键字实参,都是传给函数eat的
g1=gevent.spawn(func,1,,2,3,x=4,y=5)

g2=gevent.spawn(func2)

g1.join() #等待g1结束

g2.join() #等待g2结束

#或者上述两步合作一步：gevent.joinall([g1,g2])

g1.value#拿到func1的返回值
'''
import gevent
import time

def eat(name):
    print(f"{name} eat 1")
    gevent.sleep(3) #* 遇到IO阻塞时会自动切换任务
    print(f"{name} eat 2")

def play(name):
    print(f"{name} play 1")
    gevent.sleep(4)
    print(f"{name} play 2")

g1=gevent.spawn(eat,"egon")
g2=gevent.spawn(play,"alex")

g1.join()
g2.join()

egon eat 1
alex play 1
egon eat 2
alex play 2


## 如果是其他模块的IO操作,不会自动切换

In [5]:
import gevent
import time

def eat(name):
    print(f"{name} eat 1")
    time.sleep(3) #! 如果是time.sleep() 不会自动切换
    print(f"{name} eat 2")

def play(name):
    print(f"{name} play 1")
    time.sleep(4)
    print(f"{name} play 2")

g1=gevent.spawn(eat,"egon")
g2=gevent.spawn(play,"alex")

g1.join()
g2.join()

egon eat 1
egon eat 2
alex play 1
alex play 2


## 解决办法,引入monkey
上例gevent.sleep(2)模拟的是gevent可以识别的io阻塞,

而time.sleep(2)或其他的阻塞,gevent是不能直接识别的需要用下面一行代码,打补丁,就可以识别了

from gevent import monkey;monkey.patch_all()必须放到被打补丁者的前面，如time，socket模块之前

或者我们干脆记忆成：要用gevent，需要将from gevent import monkey;monkey.patch_all()放到文件的开头

In [1]:
import gevent
from gevent import monkey
monkey.patch_all()  # * 对所有IO操作打补丁
import time

def eat(name):
    print(f"{name} eat 1")
    time.sleep(3) 
    print(f"{name} eat 2")

def play(name):
    print(f"{name} play 1")
    time.sleep(4)
    print(f"{name} play 2")

g1=gevent.spawn(eat,"egon")
g2=gevent.spawn(play,"alex")

g1.join()
g2.join()

egon eat 1
alex play 1
egon eat 2
alex play 2


d:\python\Anaconda3\envs\study\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


##  我们可以用threading.current_thread().getName()来查看每个g1和g2，查看的结果为DummyThread-n，即假线程

In [1]:
from threading import current_thread
import gevent
from gevent import monkey
monkey.patch_all()  # * 对所有IO操作打补丁
import time

def eat(name):
    print(f"{name} eat 1")
    time.sleep(3)
    print(f"{name} eat 2")
    print(f"eat threadname is {current_thread().getName()}")

def play(name):
    print(f"{name} play 1")
    time.sleep(4)
    print(f"{name} play 2")
    print(f"play threadname is {current_thread().getName()}")

g1=gevent.spawn(eat,"egon")
g2=gevent.spawn(play,"alex")

gevent.joinall([g1,g2])

egon eat 1
alex play 1
egon eat 2
eat threadname is Dummy-3
alex play 2
play threadname is Dummy-4


[<Greenlet at 0x22c08faa2c0: _run>, <Greenlet at 0x22c091355e0: _run>]

d:\python\Anaconda3\envs\study\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
